In [60]:
import typing as t
from typing import List
from functools import reduce
import numpy as np
import torch
import random
from tabulate import tabulate
from torch import Tensor

In [61]:
MOONS = ['🌑', '🌒', '🌓', '🌔', '🌕', '🌘', '🌗', '🌖', '🌕']

In [62]:

# Function to initialize the board with starting pieces
def initialize_board():
    board = torch.zeros(8, 8, dtype=torch.long)
    x = torch.randint(7, size=(4,))
    board[3, 3] = -1
    board[3, 4] = 1
    board[4, 3] = 1
    board[4, 4] = -1
    return board



In [154]:
torch.randint(7, size=(4,))

tensor([1, 0, 4, 5])

In [258]:
def get_random_board() -> torch.Tensor:
    i, j = random.randint(0, 20), random.randint(0, 20)
    board = torch.zeros(8, 8, dtype=torch.long)
    x1 = torch.randint(7, size=(i,))
    y1 = torch.randint(7, size=(i,))
    x2 = torch.randint(7, size=(j,))
    y2 = torch.randint(7, size=(j,))

    for x, y in zip(x1, y1):
        board[x, y] = 1
    for x, y in zip(x2, y2):
        board[x, y] = -1
    return board

In [242]:
board = initialize_board()

In [218]:
board

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, -1,  1,  0,  0,  0],
        [ 0,  0,  0,  1, -1,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [65]:
moves = torch.vstack(torch.where(board == -1))
moves.T

tensor([[3, 3],
        [4, 4]])

In [66]:
torch.vstack(torch.where(board == -1))[:, 0]

tensor([3, 3])

In [67]:
def get_player_pieces(board: torch.Tensor, side: t.Union[int, t.Tuple[int,...]] = 1) -> torch.Tensor:
    """ Function to select squares by the value assigned to them.
    E.g. select squares where blacks stay. """

    if type(side) == int:
        moves = torch.vstack(torch.where(board == side))

    elif type(side) in (list, tuple):
        # Apply each value step by step
        conditions = [board == s for s in side]

        # Combine all conditions to one
        conditions = reduce(torch.logical_or, conditions)
        moves = torch.vstack(torch.where(conditions))
    else: raise TypeError

    return moves.T  # By trasponing the matrix, we turn [(x1, x2), (y1, y2)] into [(x1, y1),

In [68]:
get_player_pieces(board, side=(1)).tolist()

[[3, 4], [4, 3]]

In [69]:
board[(2,3)] = 1
board[(3,3)] = 0

In [259]:
def get_all_possible_moves(board: torch.Tensor):
    """ We will get a list of all 'valid' the cells that we could move to. Conditions:
    1) These cells are not occupied by any player;
    2) At least ONE of Neighboring cells must be occupied
    (as we cannot place a chip on the edge of the board at the beginning of the game);
    3) Squares are within the borders of the board
    """
    valid_moves = set()
    all_occupied_squares = get_player_pieces(board, side=(1,-1)).tolist()
    all_available_squares = get_player_pieces(board, side=0).tolist()  # First condition
    for row, col in all_available_squares:
        if (row, col) in all_occupied_squares:
            continue
        # Second condition - check for neighbor squares
        neighbors = torch.tensor(
            [
                [row - 1, col - 1], [row - 1, col], [row - 1, col + 1],
                [row, col - 1],                         [row, col + 1],
                [row + 1, col - 1], [row + 1, col], [row + 1, col + 1],

            ]
        )

        # Check for Condition №3
        neighbors = (
            neighbors[
                (neighbors[:, 0] >= 0) &
                (neighbors[:, 0] < 8) &
                (neighbors[:, 1] >= 0) &
                (neighbors[:, 1] < 8)
            ]
        ).tolist()
        if any([cell in all_occupied_squares for cell in neighbors]):
            valid_moves.add((row, col))
        # for cell in neighbors:
        #     if cell in all_occupied_squares:
        #         # Below we transform Tensor to tuple to make it strictly deterministic
        #         # (two tensors with same value might have different hashes)
        #         valid_moves.add((row, col))
    return sorted(tuple(valid_moves))

In [236]:
def get_valid_moves(moves: torch.Tensor) -> torch.Tensor:
    """ After we got list of possible moves, we validate them to get the
     list of moves which we are actually allowed to do.
     That is where the basic rule of the Reversi game takes effect. """
    pass

In [243]:
board = get_random_board()

i is 19; j is 9
tensor([[ 0, -1,  1,  0,  0, -1,  0,  0],
        [ 1,  0,  0,  0, -1,  1,  0,  0],
        [ 0,  0,  0, -1,  1,  0,  0,  0],
        [ 1, -1,  1,  0,  0,  0, -1,  0],
        [ 0,  1, -1,  0,  0,  1,  0,  0],
        [-1,  0,  1,  1,  1,  0,  0,  0],
        [ 0,  0, -1,  1,  1,  0,  1,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0]])


In [248]:
get_all_possible_moves(board)

[(0, 0),
 (0, 3),
 (0, 4),
 (0, 6),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 6),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 5),
 (2, 6),
 (2, 7),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 7),
 (4, 0),
 (4, 3),
 (4, 4),
 (4, 6),
 (4, 7),
 (5, 1),
 (5, 5),
 (5, 6),
 (5, 7),
 (6, 0),
 (6, 1),
 (6, 5),
 (6, 7),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7)]

In [213]:
# a = torch.tensor(list(a), dtype=torch.long)

In [260]:
def show_board(board: torch.Tensor, first_player_pieces, second_player_pieces):
    # board  = board.detach().clone().numpy().astype(str)
    # print(torch.gather(board, index=torch.where((board[:, 0] >= 0) & (board[:, 0] < 8))))

    available_squares = get_all_possible_moves(board)
    board = np.full((8, 8), fill_value='🟩', dtype=str)
    for p in first_player_pieces:
        board[*p] = '🌑'
    for p in second_player_pieces:
        board[*p] = '🌕'

    for p in available_squares:
        board[*p] = '🟧'

    board = tabulate(board, showindex=True, headers=["/", "a", "b", "c", "d", "e", "f", "g", "h"],
                                  tablefmt="tight"
    )
    print(board)

In [288]:
board = get_random_board()
show_board(
    board,
    get_player_pieces(board, side=(1,)),
    get_player_pieces(board, side=(-1))
    )

  /  a    b    c    d    e    f    g    h
---  ---  ---  ---  ---  ---  ---  ---  ---
  0  🌕   🌕   🟧   🟧   🟧   🌑   🟧   🟧
  1  🟧   🟧   🟧   🟧   🌑   🌕   🌑   🟧
  2  🟩   🟧   🌑   🟧   🟧   🟧   🌑   🟧
  3  🟧   🟧   🌑   🟧   🟧   🌕   🟧   🟧
  4  🟧   🌑   🟧   🌑   🌕   🌑   🟧   🟧
  5  🌕   🟧   🟧   🟧   🌑   🟧   🌕   🟧
  6  🟧   🟧   🟩   🟧   🌕   🟧   🟧   🟧
  7  🟩   🟩   🟩   🟧   🟧   🟧   🟩   🟩


In [256]:
torch.tensor([7, 4]) in torch.tensor([[2, 3], [3, 4], [4, 3], [4, 4]]).tolist()

False

In [34]:
def validate_move(move: t.Tuple[int, int], board: torch.Tensor):
    # print(tabulate(board, showindex=True, headers=["/", "a", "b", "c", "d", "e", "f", "g", "h"],
    #                  tablefmt="pretty")
    # )
    row, col = move
    print(move, board[move].item())

    t = torch.tensor(
        [

            [row + 1, col], [row - 1, col],


            [row, col + 1], [row, col - 1],

        ]
    )
    print(t)
    # t = t[t < 8]

    # Exclude the moves that goes beyond the borders
    t = t[(t[:, 0] < 8) & (t[:, 1] < 8)]

    print(t)
    # print(torch.split(t, 2))
    # print(t, t[t < 8].reshape(-1, 2), sep='\n')
    # print(board[t])
    # print(
    #     torch.any(
    #         t < 8, dim=-1,
    #     ).shape
    # )
    # print(board[move].item())

In [35]:
print(get_player_pieces(board, side=0))
torch.tensor([7, 1]) in get_player_pieces(board, side=0)

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [1, 0],
        [1, 1],
        [1, 2],
        [1, 3],
        [1, 4],
        [1, 5],
        [1, 6],
        [1, 7],
        [2, 0],
        [2, 1],
        [2, 2],
        [2, 3],
        [2, 4],
        [2, 5],
        [2, 6],
        [2, 7],
        [3, 0],
        [3, 1],
        [3, 2],
        [3, 5],
        [3, 6],
        [3, 7],
        [4, 0],
        [4, 1],
        [4, 2],
        [4, 5],
        [4, 6],
        [4, 7],
        [5, 0],
        [5, 1],
        [5, 2],
        [5, 3],
        [5, 4],
        [5, 5],
        [5, 6],
        [5, 7],
        [6, 0],
        [6, 1],
        [6, 2],
        [6, 3],
        [6, 4],
        [6, 5],
        [6, 6],
        [6, 7],
        [7, 0],
        [7, 1],
        [7, 2],
        [7, 3],
        [7, 4],
        [7, 5],
        [7, 6],
        [7, 7]])


True

In [36]:


# Function to make a move
def make_move(board, row, col, player):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    opponent = -1 if player == 1 else 1
    # validate_move((row, col), board)
    # pieces = get_player_pieces(board, side=0)
    # print(f"CHECK: {torch.tensor((row, col)) in pieces}")
    board[row, col] = player
    for dx, dy in directions:
        x, y = row + dx, col + dy
        flipped = []
        while 0 <= x < 8 and 0 <= y < 8 and board[x, y] == opponent:
            flipped.append((x, y))
            x += dx
            y += dy
        if 0 <= x < 8 and 0 <= y < 8 and board[x, y] == player:
            for fx, fy in flipped:
                board[fx, fy] = player
    return board

In [37]:

# Function to check if the game is over
def is_game_over(board):
    return not torch.any(board == 0)

In [42]:

# Main game loop
def play_game():
    board = initialize_board()
    # print(board)
    show_board(board, get_player_pieces(board, side=(1,)), get_player_pieces(board, side=(-1)) )
    player = 1
    while not is_game_over(board):
        # print_board(board)
        show_board(board, get_player_pieces(board, side=(1,)), get_player_pieces(board, side=(-1)) )
        if player == 1:
            valid_moves = []
            for i in range(8):
                for j in range(8):
                    if board[i, j] == 0:
                        valid_moves.append((i, j))
            if valid_moves:
                row, col = random.choice(valid_moves)
                board = make_move(board, row, col, player)
        else:
            print("Computer's turn...")
            valid_moves = []
            for i in range(8):
                for j in range(8):
                    if board[i, j] == 0:
                        valid_moves.append((i, j))
            if valid_moves:
                row, col = random.choice(valid_moves)
                board = make_move(board, row, col, player)
        player = -1 if player == 1 else 1

    show_board(board, get_player_pieces(board, side=(1,)), get_player_pieces(board, side=(-1)) )
    x_count = torch.sum(board == 1)
    o_count = torch.sum(board == -1)
    if x_count > o_count:
        print("Player X wins!")
    elif o_count > x_count:
        print("Player O wins!")
    else:
        print("It's a tie!")

In [43]:

# Run the game
play_game()

  /  a    b    c    d    e    f    g    h
---  ---  ---  ---  ---  ---  ---  ---  ---
  0  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  1  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  2  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  3  🟩   🟩   🟩   🌕   🌑   🟩   🟩   🟩
  4  🟩   🟩   🟩   🌑   🌕   🟩   🟩   🟩
  5  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  6  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  7  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  /  a    b    c    d    e    f    g    h
---  ---  ---  ---  ---  ---  ---  ---  ---
  0  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  1  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  2  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  3  🟩   🟩   🟩   🌕   🌑   🟩   🟩   🟩
  4  🟩   🟩   🟩   🌑   🌕   🟩   🟩   🟩
  5  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  6  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  7  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  /  a    b    c    d    e    f    g    h
---  ---  ---  ---  ---  ---  ---  ---  ---
  0  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  1  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  2  🟩   🟩   🟩   🟩   🟩   🟩   🟩   🟩
  3  🟩   🟩   🟩   🌕   🌑   🟩   🟩   🟩
  4  🟩   🟩   🟩   🌑   🌕   🟩   🟩   🟩
  5  🟩 